In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    )
from peft import LoraConfig


# Loading the dataset

Since the data was already processed into a satisfying format, we simply need to import the 'text' column.

In [6]:
df = pd.read_csv("/content/drive/MyDrive/data/generator_training_data.csv", usecols=['text'])
df.head()

,text
0,We went here with our kids for Xmas holiday an...
1,I had a wonderful time for a couple of nights....
2,"When traveling to Tuscany in winter, please kn..."
3,The hotel is new and very neat. Comfortable ro...
4,The hotel consists of three sites located on a...


# Fine-tuning Llama 2

Norouzi (2023) describes an efficient way to handle Llama 2 despite the system RAM limitations that Google Colab comes with and was, therefore, used as the main source throughout this section. For more details see: [Mastering Llama 2: A Comprehensive Guide to Fine-Tuning in Google Colab](https://artificialcorner.com/mastering-llama-2-a-comprehensive-guide-to-fine-tuning-in-google-colab-bedfcc692b7f)

In [7]:
# Declaring the model name; as described in the paper, the 7 billion parameter version of Llama 2 is used
model_name = "meta-llama/Llama-2-7b-hf"

The parameters for the quantization config need to be defined:

In [8]:
# Load the entire model on the GPU 0
device_map = {"": 0}

# Set base model loading in 4-bits
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4); nf4 is shown to be better in the QLoRA paper
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [9]:
# Fetch the appropriate datatype from the torch library
compute_dtype= getattr(torch, bnb_4bit_compute_dtype)

# Initialize the quantization config using the previously declared parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

Now that we got both the `device_map` set to one GPU and the quantization config `bnb_config` initialized, the base model can be loaded into the system without any memory issues:

In [10]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

As the original tokenizer has `pad_id` set to -1, we need to to define a custom padding token. While the [Llama 2 huggingface documentation](https://huggingface.co/docs/transformers/main/model_doc/llama2#overview) recommends using `tokenizer.add_special_tokens({"pad_token":"<pad>"})`, Norouzi points out that this can introduce CUDA-related errors. Therefore, he sees directly setting the `pad_token` as a safer option.




In [23]:
# Define a custom padding token
tokenizer.pad_token = "<PAD>"

# Set the padding direction to the right
tokenizer.padding_side = "right"

These paramaters are set in accordance to the QLoRA metod.

In [24]:
# LoRA attention dimension
lora_r = 64
# Alpha for LoRA scaling
lora_alpha = 64
# Dropout probability for LoRA
lora_dropout = 0.1

In [25]:
# Create the LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    inference_mode=False,
    bias="none",
    task_type="CAUSAL_LM",
)